<a href="https://colab.research.google.com/github/HumayDS/Big-data-analysis/blob/main/Lesson_9___regression_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import pandas as pd

In [74]:
df  = pd.read_csv('https://raw.githubusercontent.com/HumayDS/Big-data-analysis/main/tip.csv')
df.head()

,Unnamed: 0,total_bill,tip,sex,smoker,day,time,size
0,0,16.99,1.01,Female,No,Sun,Dinner,2
1,1,10.34,1.66,Male,No,Sun,Dinner,3
2,2,21.01,3.50,Male,No,Sun,Dinner,3
3,3,23.68,3.31,Male,No,Sun,Dinner,2
4,4,24.59,NaN,Female,No,Sun,Dinner,4


In [75]:
df = df.dropna()
df = df.drop('Unnamed: 0' , axis =1 )

In [78]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_bill,241.0,19.784564,8.950151,3.07,13.28,17.78,24.08,50.81
tip,241.0,3.490415,4.665933,1.00,2.00,2.92,3.71,50.00
size,241.0,2.560166,0.947315,1.00,2.00,2.00,3.00,6.00


In [79]:
df = df[df.tip < 36]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_bill,238.0,19.746345,8.946879,3.07,13.2725,17.685,24.0750,50.81
tip,238.0,2.996597,1.399192,1.00,2.0000,2.855,3.5775,10.00
size,238.0,2.567227,0.951180,1.00,2.0000,2.000,3.0000,6.00


In [80]:
df.corr()

<ipython-input-80-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,total_bill,tip,size
total_bill,1.000000,0.677298,0.610208
tip,0.677298,1.000000,0.497082
size,0.610208,0.497082,1.000000


In [81]:
#Create dummy values
dmy = pd.get_dummies(df.select_dtypes('object') ,  drop_first = True)
dmy.head()

,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
0,0,0,0,1,0,0
1,1,0,0,1,0,0
2,1,0,0,1,0,0
3,1,0,0,1,0,0
5,1,0,0,1,0,0


In [82]:
#Select categoric columns
cat_columns = df.select_dtypes('object')

In [83]:
#Select numeric columns
num_cols = df.drop(cat_columns , axis = 1)

In [84]:
#Concatenate numeric cols with dummies
df_cleaned = pd.concat([num_cols , dmy] , axis= 1)
df_cleaned.head()

,total_bill,tip,size,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
0,16.99,1.01,2,0,0,0,1,0,0
1,10.34,1.66,3,1,0,0,1,0,0
2,21.01,3.50,3,1,0,0,1,0,0
3,23.68,3.31,2,1,0,0,1,0,0
5,25.29,4.71,4,1,0,0,1,0,0


In [85]:
#Creating test train dataset
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict


In [86]:
#Creating test train dataset
y = df_cleaned['tip']
X = df_cleaned.drop('tip' , axis = 1)
import statsmodels.api as sm
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

In [97]:
#Building simple stats linear reg model
import statsmodels as sm
import statsmodels.regression.linear_model as sm
lm = sm.OLS(y_train, X_train)
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     24.55
Date:                Sat, 04 Nov 2023   Prob (F-statistic):           8.68e-25
Time:                        10:27:16   Log-Likelihood:                -252.72
No. Observations:                 178   AIC:                             523.4
Df Residuals:                     169   BIC:                             552.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8934      0.418      2.136      0.034       0.068       1.719
total_bill     0.0985      0.011      8.702      0.000       0.076       0.121
size           0.2423      0.109      2.215      0.028       0.026       0.458
sex_Male      -0.0204      0.166     -0.123      0.903      -0.349       0.308
smoker_Yes    -0.2505      0.174     -1.439      0.152      -0.594       0.093
day_Sat       -0.3818      0.371     -1.028      0.306      -1.115       0.352
day_Sun       -0.3200      0.389     -0.823      0.412      -1.088       0.448
day_Thur      -0.3261      0.430     -0.759      0.449      -1.174       0.522
time_Lunch    -0.0209      0.497     -0.042      0.967      -1.001       0.960
==============================================================================
Omnibus:                       18.336   Durbin-Watson:                   2.250
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.679
Skew:                           0.463   Prob(JB):                     6.58e-09
Kurtosis:                       5.055   Cond. No.                         208.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [88]:
predicted = model.predict(X_train)
predicted

130    2.889264
124    2.259819
69     2.203278
223    3.172877
70     2.159374
         ...   
111    1.467775
15     3.162441
96     3.793131
185    3.801811
106    2.742829
Length: 178, dtype: float64

In [89]:
y_train

130    1.50
124    2.52
69     2.09
223    3.00
70     1.97
       ... 
111    1.00
15     3.92
96     4.00
185    5.00
106    4.06
Name: tip, Length: 178, dtype: float64

In [96]:
#RMSE Train xetasi -- BIAS
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

rmse = np.sqrt(mean_squared_error(y_train, predicted))
rmse

1.0008563753465551

In [ ]:
model.predict(X_test)

In [94]:
#RMSE for test set ---Test xetasi
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

1.0829143581515448

Create new model

In [105]:
df  = pd.read_csv('https://raw.githubusercontent.com/HumayDS/Big-data-analysis/main/tip.csv')
df = df.dropna()
df = df.drop('Unnamed: 0' , axis =1 )
df = df[df.tip < 36]
df = df[['tip','total_bill','size']]
#Creating test train dataset
y = df['tip']
X = df.drop('tip' , axis = 1)
import statsmodels.api as sm
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

In [107]:
#Building simple stats linear reg model
import statsmodels as sm
import statsmodels.regression.linear_model as sm
lm = sm.OLS(y_train, X_train)
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.523
Method:                 Least Squares   F-statistic:                     98.14
Date:                Sat, 04 Nov 2023   Prob (F-statistic):           2.61e-29
Time:                        10:32:08   Log-Likelihood:                -254.41
No. Observations:                 178   AIC:                             514.8
Df Residuals:                     175   BIC:                             524.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4689      0.220      2.129      0.035       0.034       0.904
total_bill     0.0942      0.011      8.895      0.000       0.073       0.115
size           0.2734      0.104      2.627      0.009       0.068       0.479
==============================================================================
Omnibus:                       13.422   Durbin-Watson:                   2.250
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               28.406
Skew:                           0.287   Prob(JB):                     6.79e-07
Kurtosis:                       4.871   Cond. No.                         66.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [108]:
#RMSE Train xetasi -- BIAS
predicted = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, predicted))
rmse

1.010372801945225

In [109]:
#RMSE for test set ---Test xetasi
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

1.0486291528899079